In [1]:
library(dplyr)
library(tidyr)
library(ggplot2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union






***Figure 5E- plotting (iRegulon calculated previously)***



In [2]:
setwd('/home/jovyan/data/bone_marrow/data')
auc_mtx<-read.csv('auc_mtx_bm_equal_healthy_downs_210207.csv', row.names = 1, check.names = FALSE)

In [3]:
#import full dataset metadata to add clustering information
setwd('/home/jovyan/data/bone_marrow/raw_data')
meta<-read.csv("meta.csv", row.names = 1)
meta<-select(meta, clusters_jan, Status)

In [4]:
auc_mtx$join<-rownames(auc_mtx)
meta$join<-rownames(meta)

auc_mtx<-left_join(auc_mtx, meta, by = "join")
rownames(auc_mtx)<-auc_mtx$join

auc_mtx$join<-NULL

In [5]:
#reorder columns so clustering and Status are the first two columns

test<-colnames(auc_mtx)
test<-test[test != "Status"]
test<-test[test != "clusters_jan"]
auc_mtx <- auc_mtx[, c("Status", "clusters_jan", test)]

In [6]:
#perform t-testing for Down's AUC scores vs non-Down's AUC scores

df<-data.frame("p_value" = character(), "direction" = character(), "gene" = character(), "celltype" = character())

for(i in unique(auc_mtx$clusters_jan)){
    subset_by_cluster<-dplyr::filter(auc_mtx, clusters_jan == i)
    
    #calculate for Down's
    t_tested_pvalue<-sapply(subset_by_cluster[,3:length(colnames(subset_by_cluster))], 
                   function(g) (t.test(g ~ subset_by_cluster$Status)$p.value))
    t_tested_direction<-sapply(subset_by_cluster[,3:length(colnames(subset_by_cluster))], 
                   function(g) (t.test(g ~ subset_by_cluster$Status)$statistic))
    t_test<-data.frame(p_value = t_tested_pvalue, direction = t_tested_direction)
    t_test$direction <- ifelse(t_test$direction < 0, "Healthy","Downs")
    t_test$gene <- rownames(t_test)
    t_test$celltype<-i
    df<-rbind(df, t_test)
                                            
}

In [7]:
test$gene <- factor(test$gene, level = append(test_downs$gene, test_healthy$gene))

ERROR: Error in test$gene: $ operator is invalid for atomic vectors


In [342]:
for(i in unique(df$celltype)){
    test<-dplyr::filter(df, celltype == i)
    
    test$neg_log_p_value<- -log10(test$p_value)
    
    test_downs<-subset(test, direction == "Downs")
    test_downs<-test_downs %>% arrange(p_value)
    test_downs<-head(test_downs, 15)
    
    test_healthy<-subset(test, direction == "Healthy")
    test_healthy<-test_healthy %>% arrange(p_value)
    test_healthy<-head(test_healthy, 15)

    test<-rbind(test_downs, test_healthy)

    test$gene <- factor(test$gene, level = append(test_downs$gene, rev(test_healthy$gene)))

    ggplot(test, aes(x=gene, y=neg_log_p_value, label=neg_log_p_value)) + 
  geom_bar(stat='identity', aes(fill=direction), width=.5)  +
  scale_fill_manual(values = c("Healthy"="#c3dfe0", "Downs"="#ae0000"))+
  theme_bw() + theme(panel.grid.major = element_blank(), 
                     panel.grid.minor = element_blank(),
                    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 10))+ 
  labs(title= paste0(i, "_regulons"))
   setwd('/home/jovyan/data/bone_marrow/figures')
    ggsave(paste0(i, "_regulons.pdf"), plot = last_plot(), device = NULL, path = NULL,  scale = 1, dpi = 300)
                                            
}

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

